In [2]:
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Load the data
years = list(range(1980, 2025))
dfs = []

# Assuming all the CSV files are in the format 'daily_aqi_by_county_<year>.csv'
for year in years:
    file_name = f"daily_aqi_data/daily_aqi_by_county_{year}.csv"
    df = pd.read_csv(file_name)
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year

state_yearly_aqi = data.groupby(["State Name", "Year"])["AQI"].mean().reset_index()

state_yearly_aqi.head()

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_76347/1649178331.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_76347/1649178331.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_76347/1649178331.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_76347/1649178331.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_76347/1649178331.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on im

,State Name,Year,AQI
0,Alabama,1980,58.110709
1,Alabama,1981,54.684581
2,Alabama,1982,50.273892
3,Alabama,1983,56.911208
4,Alabama,1984,55.326143


In [ ]:
# Plot the aqi trends over time with event annotations
state_yearly_aqi.rename(columns={"State Name": "state", "Year": "year"}, inplace=True)

aqi_trends = state_yearly_aqi.groupby("year", as_index=False)["AQI"].mean()

policy_years = [1987, 1990, 2005, 2008, 2011, 2015, 2020, 2023]
policy_labels = [
    f"Montreal Protocol ({policy_years[0]})",
    f"Clean Air Act Amendments ({policy_years[1]})",
    f"Energy Policy Act ({policy_years[2]})",
    f"2008 Economic Recession ({policy_years[3]})",
    f"Cross-State Air Pollution Rule (CSAPR) ({policy_years[4]})",
    f"Clean Power Plan ({policy_years[5]})",
    f"COVID-19 Pandemic ({policy_years[6]})",
    f"Canadian Wildfires ({policy_years[7]})",
]

policy_aqi = [
    aqi_trends.loc[aqi_trends["year"] == year, "AQI"].values[0] for year in policy_years
]

policy_data = pd.DataFrame(
    {"year": policy_years, "policy": policy_labels, "AQI": policy_aqi}
)

policy_data["adjusted_AQI"] = [
    aqi + 5 if i % 2 == 0 else aqi - 6 for i, aqi in enumerate(policy_data["AQI"])
]

line_chart = (
    alt.Chart(aqi_trends)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y(
            "AQI:Q",
            title="Average AQI",
            scale=alt.Scale(
                domain=[aqi_trends["AQI"].min() - 5, aqi_trends["AQI"].max() + 2]
            ),
        ),
    )
    .properties(
        width=800,
        height=400,
        title="AQI Trends Over Time with Significant Environmental Events",
    )
)

policy_markers = (
    alt.Chart(policy_data)
    .mark_point(size=100, color="red")
    .encode(
        x=alt.X("year:O"),
        y=alt.Y("AQI:Q"),
    )
)

policy_labels_chart = (
    alt.Chart(policy_data)
    .mark_text(
        align="center",
        baseline="middle",
    )
    .encode(
        x=alt.X("year:O"),
        y=alt.Y("adjusted_AQI:Q"),
        text=alt.Text("policy"),
    )
)

connector_lines = (
    alt.Chart(policy_data)
    .mark_rule(color="gray", strokeDash=[2, 2])
    .encode(
        x=alt.X("year:O"),
        y=alt.Y("AQI:Q"),
        y2=alt.Y2("adjusted_AQI:Q"),
    )
)

interactive_chart = line_chart + policy_markers + policy_labels_chart + connector_lines

interactive_chart

alt.LayerChart(...)